In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import SubplotSpec
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

In [ ]:
def mask_singularities(data,cutoff = 2):
    '''
    If data points have a value bigger than the cutoff, it is replaced by the average value of their neighbours
    '''
    for i, col in enumerate(data):
        for j, e in enumerate(col):
            if abs(e)>cutoff:
                if i == 0:
                    if j == 0:
                        data[i][j] = np.average([data[i][j+1],
                                                 data[i+1][j],data[i+1][j+1]])
                        continue
                    elif j == len(col)-1:
                        data[i][j] = np.average([data[i][j-1],
                                                 data[i+1][j-1],data[i+1][j]])
                        continue
                    else:
                        data[i][j] = np.average([data[i][j-1],data[i][j+1],
                                                 data[i+1][j-1],data[i+1][j],data[i+1][j+1]])

                elif i == len(data)-1:
                    if j == 0:
                        data[i][j] = np.average([data[i-1][j],data[i-1][j+1],
                                                 data[i][j+1]])
                        continue
                    elif j == len(col)-1:
                        data[i][j] = np.average([data[i-1][j-1],data[i-1][j],
                                                 data[i][j-1]])
                    else:
                        data[i][j] = np.average([data[i-1][j-1],data[i-1][j],data[i-1][j+1],
                                                 data[i][j-1],data[i][j+1]])
                elif j == 0:
                    data[i][j] = np.average([data[i-1][j],data[i-1][j+1],
                                             data[i][j+1],
                                             data[i+1][j],data[i+1][j+1]])
                elif j == len(col)-1:
                    data[i][j] = np.average([data[i-1][j-1],data[i-1][j],
                                             data[i][j-1],
                                             data[i+1][j-1],data[i+1][j]])
                else:
                    data[i][j] = np.average([data[i-1][j-1],data[i-1][j],data[i-1][j+1],
                                             data[i][j-1],data[i][j+1],
                                             data[i+1][j-1],data[i+1][j],data[i+1][j+1]])
    return data

def mirror_quadrant(data,n_shift,n_mag,odd=True):
    """
    mirror first quadrant
    """
    z = np.zeros((2*n_mag-1, 2*n_shift-1))
    for i in range(2*n_mag):
        for j in range(2*n_shift):
            if i < n_mag and j < n_shift:
                z[i, j] = (-1)**odd * data[n_mag-1-i, n_shift-1-j]
            elif i > n_mag and j < n_shift:
                z[i-1, j] = data[i-n_mag, j]
            elif i < n_mag and j > n_shift:
                z[i, j-1] = (-1)**odd * data[n_mag-1-i, j-n_shift]
            elif i > n_mag and j > n_shift:
                z[i-1, j-1] = data[i-n_mag, 2*n_shift-1-j]
    return z

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    if "n_shift" in data:
        if "n_mag" in data:
            df = pd.concat([df,data])

In [ ]:
display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
key = '1720457182'

## Import file and extract data

In [ ]:
outdir = './data/' + key

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, ids = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'ids']
n_shift, min_shift, max_shift = df.set_index('key').at[key,'n_shift'], df.set_index('key').at[key,'min_shift'], df.set_index('key').at[key,'max_shift']
n_mag, min_mag, max_mag = df.set_index('key').at[key,'n_mag'], df.set_index('key').at[key,'min_mag'], df.set_index('key').at[key,'max_mag']
save_spec, t1t2, u1u2, u1u2t1t2, morb = df.set_index('key').at[key,'save_spec'], df.set_index('key').at[key,'tau1tau2'], df.set_index('key').at[key,'u1u2'], df.set_index('key').at[key,'tau1tau2u1u2'], df.set_index('key').at[key,'morb']
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']

# data
shifts = np.linspace(min_shift,max_shift,n_shift)
mags = np.linspace(min_mag,max_mag,n_mag)
    
fermi = np.load(outdir+"/fermi.npy").real.reshape((n_mag,n_shift))
if t1t2 or morb:
    ch_t1t2 = np.load(outdir+"/ch_tau1tau2.npy").real.reshape((n_mag,n_shift))
if u1u2:
    ch_u1u2 = np.load(outdir+"/ch_u1u2.npy").real.reshape((n_mag,n_shift))
if u1u2t1t2:
    ch_u1u2t1t2 = np.load(outdir+"/ch_tau1tau2u1u2.npy").real.reshape((n_mag,n_shift))
if morb:
    morbTotal = np.load(outdir+"/morb.npy").real.reshape((n_mag,n_shift))
    morbLC = np.load(outdir+"/morbLC.npy").real.reshape((n_mag,n_shift)) 
    morbIC = np.load(outdir+"/morbIC.npy").real.reshape((n_mag,n_shift))
    
# Full spectrum
if save_spec:
    spec   = np.array([np.load(outdir+'/spec_'+str(i).zfill(4)+'.npy') for i in range(n_shift*n_mag)])

In [ ]:
fig, ax = plt.subplots(2,2)
ax[0,0].imshow(fermi,interpolation='nearest',cmap='inferno')
ax[0,1].imshow(morbTotal,vmin=-0.015,vmax=0.015,interpolation='nearest',cmap='RdBu')
ax[1,0].imshow(morbLC,vmin=-0.025,vmax=0.025,interpolation='nearest',cmap='RdBu')
ax[1,1].imshow(morbIC,vmin=-0.025,vmax=0.025,interpolation='nearest',cmap='RdBu')

## Preprocessing

In [ ]:
shifts = np.linspace(0,2*np.pi,2*n_shift-1)
mags = np.linspace(-2,2,2*n_mag-1)

fermi           = mirror_quadrant(fermi,n_shift,n_mag,False)
if t1t2 or morb:
    ch_t1t2     = mirror_quadrant(ch_t1t2,n_shift,n_mag,True)
if u1u2:
    ch_u1u2     = mirror_quadrant(ch_u1u2,n_shift,n_mag,True)
if u1u2t1t2:
    ch_u1u2t1t2 = mirror_quadrant(ch_u1u2t1t2,n_shift,n_mag,False)
if morb:
    morbTotal   = mirror_quadrant(morbTotal,n_shift,n_mag,True)
    morbLC      = mirror_quadrant(morbLC,n_shift,n_mag,True)
    morbIC      = mirror_quadrant(morbIC,n_shift,n_mag,True)

In [ ]:
# adjust Fermi energy
# morbTotal = morbTotal + 2 * fermi * ch_tau1tau2.real/(-4*pi)
morbLC = morbLC - fermi * ch_t1t2.real/(-4*pi)
morbIC = morbIC - fermi * ch_t1t2.real/(-4*pi)

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20,18]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'

tfs    = 40 #title font size
lfs    = 40 #label font size
fs     = 36 #font size
cbarfs = 32 #colorbar font size

shade = 'auto'
# shade = 'gouraud'

### 2x2

In [ ]:
X, Y = np.meshgrid(mags,shifts,indexing="ij")

fig, axs = plt.subplots(2,2,figsize=(20, 18))

# fig.suptitle(tex+r': t={:.0f}, m={:.0f}, size={:.0f}, $\theta$={:.4f}, $IDS$={:.3f}, $\phi$=[{:.3f},{:.3f}], $\eta$={:.1f}'.format(t, m, size, theta, ids, phi[0], phi[1], eta), fontsize=30,y=0.99)

col1 = axs[0,0].pcolormesh(X, Y, fermi, cmap=plt.cm.inferno,shading='auto')
axs[0,0].set_title(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=tfs,y=1.02)
axs[0,0].set_xlabel(r'$m$',fontsize=lfs)
axs[0,0].set_ylabel(r'$\varphi$',fontsize=lfs)
plt.colorbar(col1,ax=axs[0,0],shrink=0.75)

col2 = axs[0,1].pcolormesh(X, Y, morbTotal, cmap=plt.cm.RdBu,shading='auto')
axs[0,1].set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs,y=1.03)
axs[0,1].set_xlabel(r'$m$',fontsize=lfs)
axs[0,1].set_ylabel(r'$\varphi$',fontsize=lfs)
cb2 = fig.colorbar(col2,ax=axs[0,1],shrink=0.75,extend='both')
cb2.cmap.set_over('black')
cb2.cmap.set_under('black')

col3 = axs[1,0].pcolormesh(X, Y, morbLC, cmap=plt.cm.RdBu,shading='auto')
axs[1,0].set_title(r'$M_{\mathrm{orb}}^{\mathrm{LC}}+\frac{E_F}{4\pi} \mathrm{Ch}_{t_1 t_2}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs,y=1.03)
axs[1,0].set_xlabel(r'$m$',fontsize=lfs)
axs[1,0].set_ylabel(r'$\varphi$',fontsize=lfs)
cb3 = fig.colorbar(col3,ax=axs[1,0],shrink=0.75,extend='both')
cb3.cmap.set_over('black')
cb3.cmap.set_under('black')

col4 = axs[1,1].pcolormesh(X, Y, morbIC, cmap=plt.cm.RdBu,shading='auto')
axs[1,1].set_title(r'$M_{\mathrm{orb}}^{\mathrm{IC}}+\frac{E_F}{4\pi} \mathrm{Ch}_{t_1 t_2}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs,y=1.03)
axs[1,1].set_xlabel(r'$m$',fontsize=lfs)
axs[1,1].set_ylabel(r'$\varphi$',fontsize=lfs)
cb4 = fig.colorbar(col4,ax=axs[1,1],shrink=0.75,extend='both')
cb4.cmap.set_over('black')
cb4.cmap.set_under('black')

for ax in axs.flatten():
    ax.set_aspect(2/np.pi,adjustable='box')
    ax.set_xticks(np.linspace(-2,2,5))
    ax.set_yticks(np.linspace(0,2*pi,5))
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    labels = [r'${:.1f} \pi$'.format(n) for n in np.linspace(0,2,5)]
    ax.set_yticklabels(labels)
    
for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.hlines(np.pi, -2, 2, color='lightgray', alpha=0.6)
    ax.vlines(0, 0, 2*np.pi, color='lightgray', alpha=0.6)

fig.tight_layout(h_pad=-10, w_pad=-0.5)
plt.savefig("./Plots/skx_MorbDiagram2x2.png", dpi=150, bbox_inches = 'tight')

In [ ]:
mag_index = 50

fig, ax = plt.subplots(4,figsize=(15,15))

for a in ax:
    a.set_xticks([n for n in np.linspace(0,2*pi,21)])
    a.xaxis.set_minor_locator(AutoMinorLocator())
    labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,21)]
    a.set_xticklabels(labels)

fig.suptitle(tex+r': t={:.0f}, m={:.0f}, size={:.0f}, $\theta$={:.4f}, ids={:.3f}, mag={:.1f}, $\phi$=[{:.3f},{:.3f}], $\eta$={:.1f}'.format(t,m,size,theta,ids,mags[mag_index],phi[0],phi[1],eta), fontsize=20,y=1)

y_ticks = np.arange(-2, 2, 0.1)

ax[0].scatter(shifts,fermi[mag_index].real,s=2);
#ax[0].set_yticks(y_ticks)
ax[0].grid(True)
#ax[0].set_ylim([0, 0.5])
ax[0].set_xlabel(r'phase shift',fontsize=20)
ax[0].set_title(r'$E_F$',fontsize=24)

y_ticks = np.arange(-4, 4, 0.5)

ax[1].scatter(shifts,morbTotal[mag_index].real,s=2);
#ax[1].set_yticks(y_ticks)
ax[1].grid(True)
#ax[1].set_ylim([-1.2, 1.2])
ax[1].set_xlabel(r'phase shift',fontsize=20)
ax[1].set_title(r'$M_{\mathrm{orb}}$',fontsize=24)

y_ticks = np.arange(-4, 4, 0.5)

ax[2].scatter(shifts,morbLC[mag_index].real,s=2);
#ax[2].set_yticks(y_ticks)
ax[2].grid(True)
#ax[2].set_ylim([-1.2, 1.2])
ax[2].set_xlabel(r'phase shift',fontsize=20)
ax[2].set_title(r'$M_{\mathrm{orb}}^{\mathrm{LC}}$',fontsize=24)

y_ticks = np.arange(-4, 4, 0.25)

ax[3].scatter(shifts,morbIC[mag_index].real,s=2);
#ax[3].set_yticks(y_ticks)
ax[3].grid(True)
#ax[3].set_ylim([0.2, 1.4])
ax[3].set_xlabel(r'phase shift',fontsize=20)
ax[3].set_title(r'$M_{\mathrm{orb}}^{\mathrm{IC}}$',fontsize=24)

plt.tight_layout()

In [ ]:
shift_index = 0

fig, ax = plt.subplots(4,figsize=(15,15))

for a in ax:
    a.set_xticks([n for n in np.linspace(-2,2,21)])
    a.xaxis.set_minor_locator(AutoMinorLocator())

fig.suptitle(tex+r': t={:.0f}, m={:.0f}, size={:.0f}, $\theta$={:.4f}, $ids$={:.3f}, shift={:.1f}, $\phi$=[{:.3f},{:.3f}], $\eta$={:.1f}'.format(t,m,size,theta,ids,shifts[shift_index],phi[0],phi[1],eta), fontsize=20,y=1)

y_ticks = np.arange(0.9, 1.1, 0.001)

ax[0].scatter(mags,fermi[:,shift_index].real,s=1);
#ax[0].set_yticks(y_ticks)
ax[0].grid(True)
#ax[0].set_ylim([0.9981, 1.0019])
#ax[0].set_ylim([1.0291, 1.0319])
ax[0].set_xlabel(r'net magnetisation',fontsize=20)
ax[0].set_title(r'IDS',fontsize=24)

y_ticks = np.arange(-4, 4, 0.25)

ax[1].scatter(mags,morbTotal[:,shift_index].real,s=1);
#ax[1].set_yticks(y_ticks)
ax[1].grid(True)
#ax[1].set_ylim([-1.2, 0.2])
#ax[1].set_ylim([-0.2, 1.2])
ax[1].set_xlabel(r'net magnetisation',fontsize=20)
ax[1].set_title(r'$M_{\mathrm{orb}}$',fontsize=24)

y_ticks = np.arange(-4, 4, 0.5)

ax[2].scatter(mags,morbLC[:,shift_index].real,s=1);
#ax[2].set_yticks(y_ticks)
ax[2].grid(True)
#ax[2].set_ylim([-0.2, 1.2])
#ax[2].set_ylim([-1.4, 2.4])
#ax[2].set_ylim([-2,2])
ax[2].set_xlabel(r'net magnetisation',fontsize=20)
ax[2].set_title(r'$M_{\mathrm{orb}}^{\mathrm{LC}}$',fontsize=24)

y_ticks = np.arange(-4, 4, 0.5)

ax[3].scatter(mags,morbIC[:,shift_index].real,s=1);
#ax[3].set_yticks(y_ticks)
ax[3].grid(True)
#ax[3].set_ylim([-0.2, 0.2])
#ax[3].set_ylim([-0.4, 1.8])
#ax[3].set_ylim([-4,4])
ax[3].set_xlabel(r'net magnetisation',fontsize=20)
ax[3].set_title(r'$M_{\mathrm{orb}}^{\mathrm{IC}}$',fontsize=24)

plt.tight_layout()

# Analytic winding number diagram

In [ ]:
net_mag1 = np.linspace(-2,np.sqrt(3)/2,10000,endpoint=False)
net_mag2 = np.linspace(np.sqrt(3)/2+0.00001,2,10000)

def winding_boundary(m):
    s1 = ( 3*(np.pi + np.arccos(m/np.sqrt(3))) )%(2*np.pi)
    s2 = ( 3*(np.pi - np.arccos(m/np.sqrt(3))) )%(2*np.pi)
    s3 = ( 2*np.pi - np.arccos(m*np.sqrt(3)) )%(2*np.pi)
    s4 = ( 4*np.pi + np.arccos(m*np.sqrt(3)) )%(2*np.pi)
    return np.array([s1,s2,s3,s4])

In [ ]:
fig, ax = plt.subplots()

for s in winding_boundary(net_mag1): ax.plot(net_mag1,s,c='k',linewidth=1.0);
for s in winding_boundary(net_mag2): ax.plot(net_mag2,s,c='k',linewidth=1.0);

## Comparison between Chern numbers and winding number

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

c = ax.pcolormesh(X, Y, morbTotal, cmap='RdBu',shading='auto')
ax.set_title(r'$M_{\mathrm{orb}}$',fontsize=tfs,y=1.02)
ax.set_xlabel(r'net magnetisation',fontsize=lfs)
ax.set_ylabel(r'phase shift',fontsize=lfs)
plt.colorbar(c,ax=ax,shrink=0.75)

for s in winding_boundary(net_mag1): ax.plot(net_mag1,s,c='k',linewidth=1.0);
for s in winding_boundary(net_mag2): ax.plot(net_mag2,s,c='k',linewidth=1.0);

ax.tick_params(axis='both', which='major', labelsize=fs)
ax.hlines(np.pi, -2, 2, color='lightgray', alpha=0.3)
ax.vlines(0, 0, 2*np.pi, color='lightgray', alpha=0.3)
    
# plt.savefig("./Plots/diagram/"+tex+"_winding_number"+key+".png", dpi=res)

In [ ]:
X, Y = np.meshgrid(mags,shifts,indexing="ij")

fig, axs = plt.subplots(2,2,figsize=(20, 18))

#fig.suptitle(tex+r': t={:.0f}, m={:.0f}, size={:.0f}, $\theta$={:.4f}, $IDS$={:.3f}, $\phi$=[{:.3f},{:.3f}], $\eta$={:.1f}'.format(t, m, size, theta, ids, phi[0], phi[1], eta), fontsize=30,y=0.99)

col1 = axs[0,0].pcolormesh(X, Y, fermi, cmap='inferno',shading='auto')
axs[0,0].set_title(r'$E_F$',fontsize=tfs,y=1.02)
axs[0,0].set_xlabel(r'net magnetisation',fontsize=lfs)
axs[0,0].set_ylabel(r'phase shift',fontsize=lfs)
plt.colorbar(col1,ax=axs[0,0],shrink=0.75)

col2 = axs[0,1].pcolormesh(X, Y, morbTotal, cmap='RdBu',shading='auto')
axs[0,1].set_title(r'$M_{\mathrm{orb}}$',fontsize=tfs,y=1.02)
axs[0,1].set_xlabel(r'net magnetisation',fontsize=lfs)
axs[0,1].set_ylabel(r'phase shift',fontsize=lfs)
plt.colorbar(col2,ax=axs[0,1],shrink=0.75)

col3 = axs[1,0].pcolormesh(X, Y, morbLC, cmap='RdBu',shading='auto')
axs[1,0].set_title(r'$M_{\mathrm{orb}}^{\mathrm{LC}}$',fontsize=tfs,y=1.02)
axs[1,0].set_xlabel(r'net magnetisation',fontsize=lfs)
axs[1,0].set_ylabel(r'phase shift',fontsize=lfs)
plt.colorbar(col3,ax=axs[1,0],shrink=0.75)

col4 = axs[1,1].pcolormesh(X, Y, morbIC, cmap='RdBu',shading='auto')
axs[1,1].set_title(r'$M_{\mathrm{orb}}^{\mathrm{IC}}$',fontsize=tfs,y=1.02)
axs[1,1].set_xlabel(r'net magnetisation',fontsize=lfs)
axs[1,1].set_ylabel(r'phase shift',fontsize=lfs)
plt.colorbar(col4,ax=axs[1,1],shrink=0.75)

for ax in axs.flatten():
    ax.set_aspect(2/np.pi,adjustable='box')
    ax.set_xticks(np.linspace(-2,2,5))
    ax.set_yticks(np.linspace(0,2*np.pi,5))
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,5)]
    ax.set_yticklabels(labels)
    
    for s in winding_boundary(net_mag1): ax.plot(net_mag1,s,c='k',linewidth=1.0);
    for s in winding_boundary(net_mag2): ax.plot(net_mag2,s,c='k',linewidth=1.0);
    
for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.hlines(np.pi, -2, 2, color='lightgray', alpha=0.3)
    ax.vlines(0, 0, 2*np.pi, color='lightgray', alpha=0.3)

fig.tight_layout()
# plt.savefig("./Plots/skx_MorbDiagram2x2_border.png", dpi=300, bbox_inches = 'tight')